In [62]:
import os
import re
import pdfplumber
import dotenv
from openai import OpenAI
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
dotenv.load_dotenv()
openai_api_key=os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone 
loader = PyPDFLoader("/Users/deantaylor/ohio_case_scrape/District_4/2013/2013-Ohio-3861.pdf")
environment = 'us-east-1-aws'
# Initialize OpenAI
client = OpenAI(api_key=openai_api_key)
MODEL = "text-embedding-ada-002"
# Initialize Pinecone
pinecone.init(api_key=PINECONE_API_KEY, environment=environment)
# Define the index name
index_name = "doc-chat"
# Create the index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536)
# Instantiate the index
index = pinecone.Index(index_name)
# Define a function to preprocess text
def preprocess_text(text):
    # Replace consecutive spaces, newlines and tabs
    text = re.sub(r'\s+', ' ', text)
    return text

def process_pdf(file_path):
    # create a loader
    loader = PyPDFLoader(file_path)
    # load your data
    data = loader.load()
    # Split your data up into smaller documents with Chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    documents = text_splitter.split_documents(data)
    # Convert Document objects into strings
    texts = [str(doc) for doc in documents]
    return texts

# Define a function to create embeddings
def create_embeddings(texts):
    embeddings_list = []
    for text in texts:
        # res = openai.Embedding.create(input=[text], engine=MODEL)
        res = client.embeddings.create(input=text, model=MODEL).data[0].embedding #this gets just the embedding array
        # res = client.embeddings.create(input=text, model=MODEL)["data"][0]["embedding"]
        embeddings_list.append(res)
    return embeddings_list

# Define a function to upsert embeddings to Pinecone
def upsert_embeddings_to_pinecone(index, embeddings, ids):
    index.upsert(vectors=[(id, embedding) for id, embedding in zip(ids, embeddings)])

# Process a PDF and create embeddings
file_path = "/Users/deantaylor/ohio_case_scrape/District_4/2013/2013-Ohio-3861.pdf"  # Replace with your actual file path
texts = process_pdf(file_path)
embeddings = create_embeddings(texts)

In [60]:
type(embeddings)
upsert_embeddings_to_pinecone(index, embeddings, [file_path])
# embeddings


In [71]:
type(texts)
# texts

list

In [ ]:
docs = loader.load()

In [66]:
from langchain.chains.question_answering import load_qa_chain
docsearch = Pinecone.from_documents(texts[0], embeddings, index_name=index_name)
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
query = "waht is the summary of this document"
docs = docsearch.similarity_search(query)
chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query)

AttributeError: 'str' object has no attribute 'page_content'

In [ ]:
query7 = "Was this a criminal or civil case?"
resp7 = chain.run(input_documents=docs, question=query7)
print(f"{query7}: {resp7}")

In [37]:
import pinecone
import os
import dotenv
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
dotenv.load_dotenv()
openai_api_key=os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
# Connect to Pinecone
index_name = "doc-chat"
from PyPDF2 import PdfReader

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PdfReader(file)
        text = ""
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
        return text


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    separators=["\n\n", "\n", " ", ""]
)

text = extract_text_from_pdf("/Users/deantaylor/ohio_case_scrape/District_4/2013/2013-Ohio-3416.pdf")

chunks = text_splitter.split_text(text[6]['text'])[:3]
chunks
# PDF path and metadata
# pdf_path = "/Users/deantaylor/ohio_case_scrape/District_4/2013/2013-Ohio-3416.pdf"
# url = "https://example.com/pdf_file.pdf"
# case_no = "12345"

# Extract text content and prepare metadata
# text_content = extract_text_from_pdf(pdf_path)
# metadata = {"url": url, "case_no": case_no}

# embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=100,
# )

# docs = text_splitter.split_documents(text_content)
# # print(docs)
# len(docs)
# # Create document with content, metadata, and required fields
# document = {
#     'id': 'pdf_id',  # Add a unique ID for the document
#     'values': text_content,  # Assign the text content to the 'values' field
#     'metadata': metadata  # Add metadata
# }
# # # print(text_content)
# # Upsert the document into Pinecone DB
# pinecone.Index(index_name).upsert([document])

# # Disconnect from Pinecone
# pinecone.deinit()
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=100,
# )

# # len(docs)
# print(type(text_content))
# docs = text_splitter.split_documents(text_content)
# print(docs)




TypeError: string indices must be integers, not 'str'

PDF FILE EXAMPLE


In [2]:
type(loader)
file_content = loader.load()
print(len(file_content))

23


In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

In [11]:
docs = text_splitter.split_documents(file_content)
# print(docs)
len(docs)
# print(docs[1])
# print(type(docs))
# print(docs)

#for every item in docs, replace '\n' with ' '
for item in docs['Document']:
    item.replace('\n', ' ')
print(docs[1])

TypeError: list indices must be integers or slices, not str

In [6]:
#ad the url and case_no to the docs list
# docs.append(url)
# docs.append(case_no)


In [8]:
meta_data = [{'url': url, 'case_no': case_no} for _ in range(len(docs))]
docs_with_metadata = [{'content': doc, 'meta': meta} for doc, meta in zip(docs, meta_data)]

docsearch = Pinecone.from_documents(docs_with_metadata, embeddings, index_name=index_name)

AttributeError: 'dict' object has no attribute 'page_content'

In [7]:
# docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
query = "What company is this document from?"
docs = docsearch.similarity_search(query)
chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query)

In [ ]:
query1 = "What does the document say about privacy rights?"
resp1 = chain.run(input_documents=docs, question=query1)
resp1

In [ ]:
query1 = "Create a summary of the key facts of the case. If a civil case this is an example, 'John Smith on 12/12/2019 went into the store and fell on a piece of fruit.  He injured his back and later had surgery.  He then filed a lawsuit against the store for negligence.  At trial, the store prevailed and John appealed resulting in this decision.'  If a criminal case, as part of the facts, ensure you list the crimes the defendant was charged with and the trial court's determination of those charges.  Display the facts in a list format one per line.  After the list of facts, determine what the reason was for the appeal.  Finally, for the last fact, summarize the decision of the court."

query2 = "Generate a summary of the legal issues involved in the case.  List the statutes, rules or regulations that were being argued about in the case.  Display the legal issues in a list format one per line.  After the list of legal issues, determine what the reason was for the appeal.  Finally, for the last legal issue, summarize the decision of the court."

query3 = "Generate a list of all the parties involved in the case.  Display the parties in a list format one per line.  After the list of parties, determine what the reason was for the appeal.  Finally, for the last party, summarize the decision of the court."

query4 = "Create a list of the procedural history of the case.  Display the procedural history in a list format one per line.  After the list of procedural history, determine what the reason was for the appeal.  Finally, for the last procedural history, summarize the decision of the court."
# query2 = "Who are the judges who decided the case?"
# query3 = "What date was this decision published?"

In [ ]:
resp1 = chain.run(input_documents=docs, question=query1)
resp2 = chain.run(input_documents=docs, question=query2)
resp3 = chain.run(input_documents=docs, question=query3)
resp4 = chain.run(input_documents=docs, question=query4)
print(f"{resp1}")
print(f"{resp2}")
print(f"{resp3}")
print(f"{resp4}")

In [ ]:
query4 = "What is the appellate case number?"
query5 = "which Judge authored the decision?"
query6 = "What are the basic facts of this case?"


resp4 = chain.run(input_documents=docs, question=query4)
resp5 = chain.run(input_documents=docs, question=query5)
resp6 = chain.run(input_documents=docs, question=query6)

print(f"{query4}: {resp4}")
print(f"{query5}: {resp5}")
print(f"{query6}: {resp6}")

In [ ]:
query7 = "Was this a criminal or civil case?"

resp7 = chain.run(input_documents=docs, question=query7)

print(f"{query7}: {resp7}")

In [ ]:
case_texts = text_splitter.split_documents(file_content)
print(len(case_texts))

In [ ]:
case_docsearch = Pinecone.from_documents(case_texts, embeddings, index_name=index)

QUERYING THE PDF DOCUMENT


In [ ]:
query = "Who are the parties to this case?"
docs = 

In [ ]:

print(type(file_content))
print(len(file_content))
# file_content[10]
pinecone.init(      
	api_key=PINECONE_API_KEY,      
	environment=PINECONE_ENV     
)      
index_name = pinecone.Index('embedder-index')
# # The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
# docsearch = Pinecone.from_documents(docs, embeddings, index_name=index)
active_indexes = pinecone.list_indexes()

In [ ]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=20
)
texts = text_splitter.split_text(file_content)

In [ ]:
text_splitter = TokenTextSplitter(
    chunk_size=100, chunk_overlap=20
)
case_texts = text_splitter.split_text(file_content)
print(len(case_texts))
type(case_texts)

In [ ]:
case_docsearch = Pinecone.from_texts(file_content, embeddings, index_name=index_name)

SPLITTING THE TEXT


In [ ]:
file_content = open('Oz_book.txt', 'r').read()
# len(file_content)

In [ ]:
text_splitter = TokenTextSplitter(
    chunk_size=100, chunk_overlap=20
)
book_texts = text_splitter.split_text(file_content)
print(len(book_texts))
type(book_texts)
# print(book_texts[3])

PINECONE AND OPENAI EMBEDDING SETUP


In [ ]:
pinecone.init(
    api_key = PINECONE_API_KEY,
    environment=PINECONE_ENV
)
index_name = 'embedder-index'
case_docsearch=Pinecone.from_texts((t.page_content for t in book_texts), index_name=index_name)